This is a script for simulating arbitrary activity and export its results.

In [120]:
%matplotlib notebook
import IPython.display
from utils_demo import *
from sys import platform
import sys
from PIL import Image
import matplotlib.pyplot as plt
import json
import rdflib
import glob
import os
import re
import copy
import time
import random
from timeout_decorator import timeout, TimeoutError

sys.path.append('../simulation')
sys.path.append('../dataset_utils/')

import numpy as np
import random
import cv2
import add_preconds
import evolving_graph.check_programs as check_programs
import evolving_graph.utils as utils

from unity_simulator.comm_unity import UnityCommunication

In [121]:
comm = UnityCommunication(timeout_wait=45)

Select scene number. 
Please see https://github.com/xavierpuigf/virtualhome/tree/master/simulation#environment .

If you want to use the existing activity data provided by VirtualHome, you can load the dataset.
Please see the below link for getting the dataset. (Note that there are many errors in the dataset.)
https://github.com/xavierpuigf/virtualhome#dataset

In [122]:
scene = 7
scene_graph = "TrimmedTestScene" + str(scene) + "_graph"
executable_program_path = "../../VirtualHome2KG/dataset/programs_processed_precond_nograb_morepreconds/executable_programs/" + scene_graph + "/*/*.txt"
executable_program_list = []
for file_path in glob.glob(executable_program_path):
    executable_program_list.append(file_path.replace("../../VirtualHome2KG/dataset/programs_processed_precond_nograb_morepreconds/executable_programs/" + scene_graph + "/", ""))

The HomeOntology is used for randomly selecting an activity.
Load HomeOntology as RDF graph.
If you do not select an activity randomly, you do not have to use the HomeOntology.

In [123]:
rdf_g = rdflib.Graph()
rdf_g.parse("../ontology/vh2kg_ontology.ttl", format="ttl")

<Graph identifier=N32ed7c977b5d46439ef5605c2fa0a4cf (<class 'rdflib.graph.Graph'>)>

In [124]:
def get_activity_from_ontology(activity_type):
    results = []
    qres = rdf_g.query(
    """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX : <http://www.owl-ontologies.com/VirtualHome.owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
select ?activity where { 
    ?activity rdfs:subClassOf :%s .
 } 
       """ % activity_type)

    result = ""
    for row in qres:
        activity = "".join(row).replace("http://www.owl-ontologies.com/VirtualHome.owl#","")
        arr = activity.split("_")
        arr[0] = arr[0].capitalize()
        activity = " ".join(arr)
        result = activity
        results.append(result)
    return results

In [125]:
def generate_list_of_steps(file_path):
    file = open(file_path, "r", encoding="utf-8")
    i = 0
    list_of_steps = []
    program_name = ""
    description = ""
    char= "<char0>"
    while True:
        line = file.readline()
        if line:
            line = line.replace("\n","")
            if i==0:
                program_name = line
            elif i==1:
                description = line
            elif line.startswith("["):
                list_of_steps.append(line)
            else:
                pass
            i+=1
        else:
            break
    return program_name, description, list_of_steps

In [126]:
def get_activity_program(category):
    #unexecutable = ["Take shower", "Take shoes off", "Wash teeth", "Wash face", "Dust", "Clean toilet", "Clean room", "Scrubbing living room tile floor is once week activity for me", "Clean mirror", "Play games", "Play on laptop", "Read on sofa"]
    unexecutable = []
    executable = []
    activities = get_activity_from_ontology(category)
    for activity_name in activities:
        if activity_name in unexecutable:
            continue
        results = [program for program in program_list if program["name"] == activity_name]
        if len(results) == 0:
            print("Nothing: " + activity_name)
        else:
            print("Success: " + activity_name)
            executable.append({"activity_name": activity_name, "results": results})
    return executable

In [127]:
data_path = "../../VirtualHome2KG/dataset/programs_processed_precond_nograb_morepreconds/withoutconds/*/*.txt"
program_list = []
for file_path in glob.glob(data_path):
    file_name = file_path.replace("../../VirtualHome2KG/dataset/programs_processed_precond_nograb_morepreconds/withoutconds/", "")
    if file_name in executable_program_list:
        program_name, description, list_of_steps = generate_list_of_steps(file_path)
        program_list.append({
            "file_name":file_name,
            "name": program_name,
            "description": description,
            "list_of_steps": list_of_steps
        })

In [128]:
executable_activity_list = []
executable_activity_dic = {}
for activity_class in ["BedTimeSleep", "EatingDrinking", "FoodPreparation", "GettingReady", "HouseArrangement", "HouseCleaning", "HygieneStyling", "Leisure", "Other", "PhysicalActivity", "SocialInteraction", "Work"]:
    executable_activity_dic[activity_class] = get_activity_program(activity_class)
#     executable_activity_list.extend(get_activity_program(activity_class))

Nothing: Go to bed
Nothing: Read them bedtime story
Success: Take nap
Nothing: Wake me up
Nothing: Bed time
Success: Go to sleep
Nothing: Put them to sleep
Success: Sleep
Success: Put alarm clock in bedroom
Nothing: Get ready for bed
Nothing: Eat donuts
Success: Get something to drink
Success: Eat snacks and drink tea
Nothing: Eat family meals
Nothing: Eat meals
Success: Eat dinner
Nothing: Eat while watching tv
Success: Drink
Nothing: Have ice coffee
Success: Have snack
Nothing: Eat on fancy china
Success: Eat cheese
Success: Eat cereal
Success: Eat
Nothing: Eat breaksfast
Nothing: Eat dessert
Nothing: Dance for table as we eat
Nothing: Have dinner
Nothing: Dining
Nothing: Eat ice cream
Nothing: Make peanut butter sandwich
Nothing: Prepare sandwich
Nothing: Fix bowl of cereal
Nothing: Bake
Nothing: Make drink
Nothing: Peel potatoes
Nothing: Make banana bread
Nothing: Defrost chicken
Nothing: Peel kiwi
Success: Cut bread
Success: Make popcorn
Nothing: Make sandwitch
Nothing: Brew coffe

In [129]:
unsupport_unity = [
    "PutOn",
    "PutOff",
    "Drop",
    "PlugIn",
    "PlugOut",
    "Sleep",
    "Wake"
]

In [130]:
def check_unsupport_action(script):
    flag = True
    for line in script:
        m = re.search(r'\[.+\]', line)
        action = m.group().replace('[', '')
        action = action.replace(']', '')
        if action in unsupport_unity:
            flag = False
            break
    return flag

In [131]:
def switchOff(g, class_names):
    for class_name in class_names:
        objects = find_nodes(g, class_name=class_name)
        for obj in objects:
            states = [x for x in obj["states"] if x != "ON"]
            if "OFF" not in states:
                states.append("OFF")
            obj["states"] = states
#             for node in g["nodes"]:
#                 if node["id"] == obj["id"]:
#                     node = obj
        return g

In [132]:
def export(activity_name, graph_state_list, activity_cnt, time_list, script, object_name_map, out_dir_name):
    state_cnt = 0
    for graph_state in graph_state_list:
        state_cnt += 1
        file_path = "graph_state_list_with_bbox_png/" + out_dir_name + "_scene" + str(scene) + "/"  + activity_name + "/" + activity_cnt + "/activityList-graph-state-" + '{0:03d}'.format(state_cnt) + ".json"
        with open(file_path, 'w') as outfile:
            json.dump(graph_state, outfile)

    with open("graph_state_list_with_bbox_png/" + out_dir_name + "_scene" + str(scene) + "/" + activity_name + "/" + activity_cnt + "/activityList-program.txt", 'w') as f:
        for s in script:
            #ここでscriptのobject_nameを置換する処理書く
            for x in object_name_map:
                if x in s:
                    s = s.replace("<" + x  + ">", "<" + object_name_map[x] + ">")
            f.write("%s\n" % s)

    with open("graph_state_list_with_bbox_png/" + out_dir_name + "_scene" + str(scene) + "/" + activity_name + "/" + activity_cnt  + "/program-description.txt", 'w') as f:
        f.write("%s\n" % activity["name"])
        f.write("%s\n" % activity["description"])
        
    time_list = [str(time) for time in time_list]
    duration = "\n".join(time_list)
    with open("graph_state_list_with_bbox_png/" + out_dir_name + "_scene" + str(scene) + "/" + activity_name + "/" + activity_cnt + "/duration.txt", 'w') as f:
        f.write(duration)

In [133]:
def update_bbox(pre_graph, current_graph):
    try:
        new_graph= copy.deepcopy(current_graph)
        
        for pre_node in pre_graph["nodes"]:
            for new_node in new_graph["nodes"]:
                if new_node["id"] == pre_node["id"]:
                    new_node["bounding_box"] = pre_node["bounding_box"]
           
    except Exception as e:
        print(e)
    
    return new_graph

In [134]:
def update_states(pre_graph, graph_state, object_id_map):
    try:
        new_graph= copy.deepcopy(pre_graph)
        
        skip = [object_id_map[x] for x in object_id_map]
        for new_node in new_graph["nodes"]:
            new_node_id = new_node["id"]
            if new_node_id in skip:
                continue
            temp_node_id = copy.deepcopy(new_node_id)
            # scriptで使用したobject_idとgraph_stateで反映されたidが異なる場合
            if str(new_node_id) in object_id_map:
                temp_node_id = object_id_map[str(new_node_id)]
            for graph_state_node in graph_state["nodes"]:
                if temp_node_id == graph_state_node["id"]:
                    new_node["states"] = graph_state_node["states"]
                    new_node["properties"] = graph_state_node["states"]
           
    except Exception as e:
        print(e)
    
    return new_graph

In [135]:
def create_object_id_map(original_script, executed_program, g):
    try:
        object_id_map = {}
        object_name_map = {}
        for original_block_str, executed_block_str in zip(original_script, executed_program):
            original_action, original_obj_names_corr, original_inst_nums_corr = augmentation_utils.parseStrBlock(original_block_str)
            executed_action, executed_obj_names_corr, executed_inst_nums_corr = augmentation_utils.parseStrBlock(executed_block_str)
            
            #object_idのmapを作る
            for original_id, executed_id in zip(original_inst_nums_corr, executed_inst_nums_corr):
                if original_id not in object_id_map:
                    if original_id != executed_id:
                        object_id_map[original_id] = int(executed_id)
            
            #object_nameのmap作る
            for original_obj_name, original_id in zip(original_obj_names_corr, original_inst_nums_corr):
                g_objects = [x for x in g["nodes"] if x["id"] == original_id]
                if len(g_objects) != 0:
                    g_obj = g_objects[0]
                    if original_obj_name != g_obj["class_name"]:
                        object_name_map[original_obj_name] = g_obj["class_name"]
    except Exception as e:
        print(e)
    return object_id_map, object_name_map

In [136]:
def delete2000(graph_state_list):
    new_graph_state_list = []
    for graph_state in graph_state_list:
        new_nodes = [x for x in graph_state["nodes"] if x["id"] < 2000]
        new_edges = [x for x in graph_state["edges"] if x["from_id"] < 2000 and x["to_id"] < 2000]
        new_graph_state_list.append({"nodes": new_nodes, "edges": new_edges})
    return new_graph_state_list

In [137]:
@timeout(40)
def time_managed_render_script(instruction, find_solution, image_width, recording, image_height, file_name_prefix, frame_rate, vis_check_object, vis_check_character, out_graph):
    success, message = comm.render_script(script=[instruction],
#                                               processing_time_limit=60,
                                              find_solution=find_solution,
                                              image_width=image_width,
                                              image_height=image_height,  
                                              #skip_animation=False,
                                              recording=True,
#                                               gen_vid=True,
                                              #save_scene_states=True,
                                              file_name_prefix=file_name_prefix,
                                              output_folder="Output/",
                                              frame_rate=frame_rate,
                                              vis_check_object=vis_check_object, 
                                              vis_check_character=vis_check_character, 
                                              out_graph=out_graph
                                             )
    return success, message

In [138]:
out_dir_name = "test20220719"

for activity_class in executable_activity_dic:
    executable_activity_list = executable_activity_dic[activity_class]
    rand_executable_activity_list = []
    print(activity_class)
    if len(executable_activity_list) >=3:
        rand_executable_activity_list = random.sample(executable_activity_list, 3)
    elif len(executable_activity_list) > 0:
        rand_executable_activity_list = random.sample(executable_activity_list, len(executable_activity_list))
    else:
        continue
#rand_executable_activity_list = random.sample(executable_activity_list, len(executable_activity_list))
    for executable_activity in rand_executable_activity_list:
        activity_list = executable_activity["results"]
        activity_name = executable_activity["activity_name"]
        activity_cnt = 0
        try:
            # if activity_name != "Relax on sofa": 
            #     continue
            os.mkdir("graph_state_list_with_bbox_png/" + out_dir_name + "_scene" + str(scene) + "/" + activity_name)
            for activity in activity_list:
                if activity_cnt > 0:
                    break
                try:
                    new_graph_state_list = []
                    time_list = []
                    comm.reset(scene-1)
                    comm.add_character('chars/Male1')
                    success, g = comm.environment_graph();
                    print(activity["file_name"])
                    script = activity["list_of_steps"]
                    print(script)
                    executable_program = []
                    for x in script:
                        executable_program.append("<char0> " + x)
                    executable_program

                    # check if supported action
                    if check_unsupport_action(script) == False:
                        continue

    #                 #sit
    #                 sit = False
    #                 for s in script:
    #                     if "Sit" in s:
    #                         sit = True
    #                         continue
    #                 if sit:
    #                     continue

                    preconds = add_preconds.get_preconds_script(script).printCondsJSON()

                    print(preconds)
                    print("switch off")
                    g = switchOff(g, ["computer", "tv", "faucet", "television"])

                    g = check_programs.translate_graph_dict_nofile(g)
                    info = check_programs.check_script(script, preconds, graph_path=None, inp_graph_dict=g)
                    message, final_state, graph_state_list, graph_dict, id_mapping, info, helper, executed_script = info
                    # if delete randomly placced objects 
                    graph_state_list = delete2000(graph_state_list)
                    print(message)
                    executed_program = []
                    for x in executed_script:
                        executed_program.append("<char0> " + re.sub("\[\d+\]","",x.__str__()).strip())

                    if message != "Script is executable":
                        continue

                    new_script = []
                    for x in script:
                        new_script.append("<char0> " + x)

                    i = 0

                    object_id_map, object_name_map = create_object_id_map(new_script, executed_program, g)
                    print("object_id_map")
                    print(object_id_map)

                    success, g = comm.environment_graph()
                    g = update_states(g, graph_state_list[i], object_id_map)
                    new_graph_state_list.append(copy.deepcopy(g))

                    for instruction in executed_program:
                        i += 1
                        start = time.time()
    #                     try:
    #                         success, g = time_managed_render_script(instruction,False,480,360,True,activity_name + str(activity_cnt), 15,False,False,False)
    #                     except TimeoutError as e:
    #                         print(e)
    #                         break;
    #                     finally:
    #                         print("掛った時間", time.time() - start)
                        success, message = comm.render_script(script=[instruction],
    #                                               processing_time_limit=60,
                                                  find_solution=False,
                                                  image_width=480,
                                                  image_height=360,  
                                                  #skip_animation=False,
                                                  recording=True,
    #                                               gen_vid=True,
                                                  #save_scene_states=True,
                                                  file_name_prefix=activity_name + str(activity_cnt),
                                                  output_folder="Output/scene" + str(scene),
                                                  frame_rate=15,
                                                  vis_check_object=False, 
                                                  vis_check_character=False, 
                                                  out_graph=False
                                                 )
                        time_list.append(time.time() - start)
                        success, g = comm.environment_graph()
                        print(str(success) + ": " + str(instruction))
                        g = update_states(g, graph_state_list[i], object_id_map)
                        new_graph_state_list.append(copy.deepcopy(g))

                    if os.path.exists("graph_state_list_with_bbox_png/" + out_dir_name + "_scene" + str(scene) + "/" + activity_name + "/" + str(activity_cnt)) == False:
                        os.mkdir("graph_state_list_with_bbox_png/" + out_dir_name + "_scene" + str(scene) + "/" + activity_name + "/" + str(activity_cnt))
                    export(activity_name, new_graph_state_list, str(activity_cnt), time_list, new_script, object_name_map,  out_dir_name)

                    activity_cnt += 1
                except Exception as e:
                    print(e.args)
        except Exception as e:
            print(e.args)

BedTimeSleep
results_text_rebuttal_specialparsed_programs_upwork_kellensecond/split11_6.txt
['[Walk] <bedroom> (1)', '[Walk] <alarm_clock> (1)', '[Find] <alarm_clock> (1)', '[Grab] <alarm_clock> (1)', '[Find] <dresser> (1)', '[Open] <dresser> (1)', '[PutBack] <alarm_clock> (1) <dresser> (1)', '[SwitchOn] <alarm_clock> (1)']
('alarm_clock',)
results_intentions_march-13-18/file509_2.txt
['[Walk] <bedroom> (1)', '[Walk] <clothes_pants> (1)', '[Find] <clothes_pants> (1)', '[Find] <clothes_pants> (1)', '[Find] <clothes_pants> (1)', '[PutOff] <clothes_pants> (1)', '[Find] <clothes_shirt> (1)', '[Find] <clothes_shirt> (1)', '[Find] <clothes_shirt> (1)', '[PutOff] <clothes_shirt> (1)', '[Find] <clothes_shirt> (2)', '[Grab] <clothes_shirt> (2)', '[PutOn] <clothes_shirt> (2)', '[Find] <clothes_pants> (2)', '[Grab] <clothes_pants> (2)', '[PutOn] <clothes_pants> (2)', '[Find] <light> (1)', '[SwitchOff] <light> (1)', '[Find] <bed> (1)', '[Lie] <bed> (1)', '[Find] <sheets> (1)', '[Pull] <sheets> (1)

True: <char0> [WALK] <dining_room> (56)
True: <char0> [WALK] <table> (129)
True: <char0> [FIND] <chair> (152)
True: <char0> [PULL] <chair> (152)
True: <char0> [SIT] <chair> (152)
True: <char0> [FIND] <food_food> (1001)
True: <char0> [TURNTO] <food_food> (1001)
True: <char0> [LOOKAT] <food_food> (1001)
True: <char0> [GRAB] <food_food> (1001)
True: <char0> [FIND] <plate> (1002)
True: <char0> [PUTBACK] <food_food> (1001) <plate> (1002)
True: <char0> [GRAB] <food_food> (1001)
True: <char0> [PUTOBJBACK] <food_food> (1001)
True: <char0> [FIND] <fork> (1000)
True: <char0> [GRAB] <fork> (1000)
True: <char0> [EAT] <plate> (1002)
results_text_rebuttal_specialparsed_programs_upwork_kellensecond/split6_10.txt
['[Walk] <kitchen> (1)', '[Walk] <fridge> (1)', '[Find] <fridge> (1)', '[Open] <fridge> (1)', '[Find] <beer> (1)', '[Grab] <beer> (1)', '[Close] <fridge> (1)', '[Walk] <living_room> (1)', '[Walk] <couch> (1)', '[Find] <couch> (1)', '[Sit] <couch> (1)', '[Drink] <beer> (1)']
[{'atreach': [['be

('execute() takes 5 positional arguments but 6 were given',)
HouseCleaning
results_intentions_march-13-18/file367_1.txt
['[Walk] <bathroom> (1)', '[Walk] <brush> (1)', '[Find] <brush> (1)', '[Grab] <brush> (1)', '[Find] <detergent> (1)', '[Grab] <detergent> (1)', '[Find] <toilet> (1)', '[PutBack] <detergent> (1) <toilet> (1)', '[Find] <toilet> (1)', '[Scrub] <toilet> (1)', '[Rinse] <toilet> (1)', '[Find] <shower> (1)', '[Scrub] <shower> (1)', '[Rinse] <shower> (1)']
[{'atreach': [['detergent', '1'], ['brush', '1']]}, {'atreach': [['toilet', '1'], ['brush', '1']]}, {'atreach': [['shower', '1'], ['brush', '1']]}, {'location': [['brush', '1'], ['bathroom', '1']]}, {'location': [['detergent', '1'], ['bathroom', '1']]}, {'location': [['toilet', '1'], ['bathroom', '1']]}, {'location': [['shower', '1'], ['bathroom', '1']]}, {'free': ['toilet', '1']}]
switch off
('execute() takes 5 positional arguments but 6 were given',)
results_text_rebuttal_specialparsed_programs_upworknturk_second/split68_

Script is executable
name 'augmentation_utils' is not defined
object_id_map
{}
True: <char0> [WALK] <home_office> (197)
True: <char0> [WALK] <coffee_table> (1000)
True: <char0> [FIND] <novel> (1001)
True: <char0> [GRAB] <novel> (1001)
True: <char0> [FIND] <couch> (246)
True: <char0> [SIT] <couch> (246)
True: <char0> [FIND] <bookmark> (1002)
True: <char0> [READ] <novel> (1001)
Other
results_text_rebuttal_specialparsed_programs_upwork_kellensecond/split3_4.txt
['[Walk] <bathroom> (1)', '[Walk] <mirror> (1)', '[Find] <mirror> (1)', '[TurnTo] <mirror> (1)', '[LookAt] <mirror> (1)']
[{'location': [['mirror', '1'], ['bathroom', '1']]}]
switch off
Script is executable
name 'augmentation_utils' is not defined
object_id_map
{}
True: <char0> [WALK] <bathroom> (11)
True: <char0> [WALK] <mirror> (1000)
True: <char0> [FIND] <mirror> (1000)
True: <char0> [TURNTO] <mirror> (1000)
True: <char0> [LOOKAT] <mirror> (1000)
results_text_rebuttal_specialparsed_programs_upworknturk_second/split66_3.txt
['[Wa

Script is executable
name 'augmentation_utils' is not defined
object_id_map
{}
True: <char0> [WALK] <home_office> (197)
True: <char0> [WALK] <desk> (243)
True: <char0> [FIND] <computer> (273)
True: <char0> [SWITCHON] <computer> (273)
True: <char0> [FIND] <chair> (244)
True: <char0> [PULL] <chair> (244)
True: <char0> [SIT] <chair> (244)
True: <char0> [TURNTO] <computer> (273)
True: <char0> [LOOKAT] <computer> (273)
True: <char0> [FIND] <mouse> (277)
True: <char0> [GRAB] <mouse> (277)
True: <char0> [PULL] <mouse> (277)
True: <char0> [TOUCH] <mouse> (277)
True: <char0> [TURNTO] <computer> (273)
True: <char0> [LOOKAT] <computer> (273)
True: <char0> [FIND] <keyboard> (275)
True: <char0> [TYPE] <keyboard> (275)
True: <char0> [TURNTO] <computer> (273)
True: <char0> [LOOKAT] <computer> (273)
True: <char0> [TYPE] <keyboard> (275)
True: <char0> [TURNTO] <computer> (273)
True: <char0> [LOOKAT] <computer> (273)
True: <char0> [TYPE] <keyboard> (275)
True: <char0> [TOUCH] <mouse> (277)
True: <char0>

In [81]:
 len(executable_activity_list)

17

In [37]:
executable_activity_list[0]

{'activity_name': 'Go to sleep',
 'results': [{'file_name': 'results_intentions_march-13-18/file509_2.txt',
   'name': 'Go to sleep',
   'description': 'I walk into my bedroom. I change out of my clothing into pajamas. I turn off the light. I lie down in my bed and pull my sheets over me. I go to sleep.',
   'list_of_steps': ['[Walk] <bedroom> (1)',
    '[Walk] <clothes_pants> (1)',
    '[Find] <clothes_pants> (1)',
    '[Find] <clothes_pants> (1)',
    '[Find] <clothes_pants> (1)',
    '[PutOff] <clothes_pants> (1)',
    '[Find] <clothes_shirt> (1)',
    '[Find] <clothes_shirt> (1)',
    '[Find] <clothes_shirt> (1)',
    '[PutOff] <clothes_shirt> (1)',
    '[Find] <clothes_shirt> (2)',
    '[Grab] <clothes_shirt> (2)',
    '[PutOn] <clothes_shirt> (2)',
    '[Find] <clothes_pants> (2)',
    '[Grab] <clothes_pants> (2)',
    '[PutOn] <clothes_pants> (2)',
    '[Find] <light> (1)',
    '[SwitchOff] <light> (1)',
    '[Find] <bed> (1)',
    '[Lie] <bed> (1)',
    '[Find] <sheets> (1)',
 

In [40]:
cnt = 0
for executable_activity in rand_executable_activity_list:
    activity_list = executable_activity["results"]
    cnt += len(activity_list)
print(cnt)

680
